In [33]:
import pandas as pd
!pip install plotnine
from plotnine import  ggplot, aes, geom_point, geom_line, labs, theme_minimal, facet_wrap, theme
import statsmodels.api as sm
import numpy as np



In [2]:
df = pd.read_csv("merge_vets.csv")
print(df.columns)

Index(['County', 'GEOID', 'anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'density', 'hisp', 'hispvap', 'nanglo',
       'nanglovap', 'official_boundary', 'pct_dem_lead', 'vap', 'votes_dem',
       'votes_rep', 'votes_total', 'state', 'total', 'area_sqmi',
       'college_enroll', 'bachelors_up_total_sum', '20_trump', '20_biden',
       '18_cruz', '18_orourke', '20_total', '18_total', 'pct_dem_lead_18',
       'pct_dem_lead_20', '18-34_sum', '35-54_sum', '55-64_sum', '65+_sum',
       'veteran_sum', 'active_duty_sum', 'anglovap_pct', 'asianvap_pct',
       'hispvap_pct', 'blackvap_pct', 'college_enroll_pct',
       'bachelors_up_total_sum_pct', '18-34_sum_pct', '35-54_sum_pct',
       '55-64_sum_pct', '65+_sum_pct', 'veteran_sum_pct',
       'active_duty_sum_pct', 'not_bachelors_pct'],
      dtype='object')


1.0
-1.0
1.0
-1.0


In [16]:
df_counties = df.groupby("County", as_index=False)[['anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'hisp', 'hispvap', 'nanglo',
       'nanglovap','vap', 'votes_dem','votes_rep', 'votes_total', 'college_enroll', 
       'bachelors_up_total_sum', '20_trump', '20_biden',
       '18_cruz', '18_orourke', '20_total', '18_total'
       ]].sum()

df_counties['anglovap_pct'] = df_counties['anglovap'] / df_counties['vap']
df_counties['asianvap_pct'] = df_counties['asianvap'] / df_counties['vap']
df_counties['hispvap_pct'] = df_counties['hispvap'] / df_counties['vap']
df_counties['blackvap_pct'] = df_counties['blackvap'] / df_counties['vap']
df_counties['college_enroll_pct'] = df_counties['college_enroll'] / df_counties['vap']
df_counties['bachelors_up_total_sum_pct'] = df_counties['bachelors_up_total_sum'] / df_counties['vap']
df_counties['pct_dem_lead'] = df_counties['votes_dem'] / df_counties['votes_total'] - df_counties['votes_rep'] / df_counties['votes_total']
df_counties['pct_dem_lead_20'] = df_counties['20_biden'] / df_counties['20_total'] - df_counties['20_trump'] / df_counties['20_total']

# density

In [17]:
print(df_counties["votes_total"].sum())

11394783


In [ ]:
# Linear regression: change in dem margin (2024-2020) vs Hispanic VAP (%)

# Load data
csv_path = 'merge_vets.csv'  # relative to project root
_df = pd.read_csv("merge_vets.csv")

var = "hispvap_pct"

# Ensure numeric columns
cols = ['pct_dem_lead', 'pct_dem_lead_20', var,'votes_total','20_total']
for c in cols:
    _df[c] = pd.to_numeric(_df[c], errors='coerce')

# Compute changes (percentage points)
df_lr = _df.copy()
df_lr = df_lr[(df_lr['votes_total'] > 50) & (df_lr['20_total'] > 50)]
df_lr = df_lr.dropna(subset=['pct_dem_lead', 'pct_dem_lead_20', var])
df_lr['delta_pp'] = (df_lr['pct_dem_lead'] - df_lr['pct_dem_lead_20']) * 100.0
# Convert share to % for readability

# Plot: scatter + linear regression line
(
    ggplot(df_lr, aes(x=var, y='delta_pp'))
    + geom_point(alpha=0.35, size=1.4, color='#2a6db0')
    + geom_smooth(method='lm', se=True, color='#d83b3b')
    + labs(
        title='Change in Dem margin (2024 - 2020) vs Hispanic VAP',
        x='Hispanic VAP (%)',
        y='Change in Dem margin (pp)'
    )
    + theme_minimal()
)


ModuleNotFoundError: No module named 'plotly'

In [51]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
!pip install nbformat



  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.27.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.2 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.2-py3-none-any.whl (24 kB)
Using cached jsonschema-4.25.1-py3-none-any.whl (90 kB)
Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl (18 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)
Using cached rpds_py-0.27.1-cp313-cp313-macosx_11_0_arm64.whl (345 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [nbformat]6/7 [nbformat]


In [54]:

# ---------- Config ----------
csv_path   = "merge_vets.csv"
x_var      = "anglovap_pct"      # X variable (e.g., Hispanic/Anglo VAP %)
county_col = "County"            # <-- set to your county column name
id_col     = "GEOID"       # <-- set to your precinct/ID column name
weight_col = "total"             # weights for WLS (change if needed)

# ---------- Load & clean ----------
df = pd.read_csv(csv_path)
num_cols = ['pct_dem_lead', 'pct_dem_lead_20', x_var, 'votes_total', '20_total', weight_col]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Filter + compute outcome (Δ Dem margin in pp)
df = df[(df['votes_total'] > 50) & (df['20_total'] > 50)]
df = df.dropna(subset=['pct_dem_lead', 'pct_dem_lead_20', x_var, weight_col])
df['delta_pp'] = (df['pct_dem_lead'] - df['pct_dem_lead_20']) * 100.0

# ---------- Fit OLS & WLS ----------
X = sm.add_constant(df[[x_var]])
y = df['delta_pp']
w = df[weight_col].clip(lower=1)  # avoid zero/neg weights

ols = sm.OLS(y, X).fit()
wls = sm.WLS(y, X, weights=w).fit()

# Prediction grid
x_grid = np.linspace(df[x_var].min(), df[x_var].max(), 200)
Xg = sm.add_constant(pd.DataFrame({x_var: x_grid}))
yhat_ols = ols.predict(Xg)
yhat_wls = wls.predict(Xg)

# ---------- Scatter (interactive) ----------
fig = px.scatter(
    df,
    x=x_var, y='delta_pp',
    opacity=0.5,
    hover_data={
        county_col: True,
        id_col: True,
        x_var: ':.2f',
        'delta_pp': ':.2f',
        weight_col: ':.0f'
    },
    labels={
        x_var: f'{x_var} (%)',
        'delta_pp': 'Change in Dem margin (pp)'
    },
    title=f'Change in Dem margin (2024 − 2020) vs {x_var} (hover for {county_col}/{id_col})'
)

# ---------- Add regression lines ----------
fig.add_trace(go.Scatter(
    x=x_grid, y=yhat_ols,
    mode='lines',
    name='Unweighted OLS',
    line=dict(width=3, color='red')
))
fig.add_trace(go.Scatter(
    x=x_grid, y=yhat_wls,
    mode='lines',
    name=f'Weighted by {weight_col}',
    line=dict(width=3, dash='dash', color='red')
))

# Prettify axes/grid
fig.update_layout(
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
)
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

# ---------- Show in notebook ----------
fig.show(renderer="browser")

# ---------- (Optional) Save as a standalone HTML you can open/share ----------
# fig.write_html("reg_scatter_interactive.html", include_plotlyjs="cdn", auto_open=True)
